## PyTortch Regression

The following example will:

* Set the input and output schemas.
* Upload a SKLearn Logistic Regression model to Wallaroo.
* Deploy a pipeline with the uploaded SKLearn model as a pipeline step.
* Perform a test inference.
* Undeploy the pipeline.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd


Please log into the following URL in a web browser:

	https://keycloak.autoscale-uat-ee.wallaroo.dev/auth/realms/master/device?user_code=PYXH-ZCNX

Login successful!


In [ ]:

wl = wallaroo.Client()

In [2]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

prefix = "pytorch-multi-input"

In [3]:
workspace = get_workspace(f"{prefix}-jch")
wl.set_current_workspace(workspace)

{'name': 'pytorch-single-input-jch', 'id': 47, 'archived': False, 'created_by': '0892876b-8b50-4541-bf29-0e570e590c01', 'created_at': '2023-06-23T02:07:54.59147+00:00', 'models': [], 'pipelines': []}

## Data & Model Creation

In [ ]:
mock_inference_data = [np.random.rand(10, 10), np.random.rand(10, 5)]
mock_dataframe = pd.DataFrame(
    {
        "input_1": mock_inference_data[0].tolist(),
        "input_2": mock_inference_data[1].tolist(),
    }
)
mock_json = mock_dataframe.to_json('data/pytorch-multi-io-data.json', orient="records")

In [4]:
input_schema = pa.schema([
    pa.field('input_1', pa.list_(pa.float64(), list_size=10)),
    pa.field('input_2', pa.list_(pa.float64(), list_size=5))
])
output_schema = pa.schema([
    pa.field('output_1', pa.list_(pa.float64(), list_size=3)),
    pa.field('output_2', pa.list_(pa.float64(), list_size=2))
])

## Upload model

In [5]:
model = wl.upload_model('pt-multi-io-model', 
                        "./models/model-auto-conversion_pytorch_multi_io_model.pt", 
                        framework=Framework.PYTORCH, 
                        input_schema=input_schema, 
                        output_schema=output_schema
                       )
model

Waiting for model conversion... It may take up to 10.0min.
Model is Pending conversion..Converting...........Ready.


{'name': 'pt-single-io-model', 'version': '8f91dee1-79e0-449b-9a59-0e93ba4a1ba9', 'file_name': 'model-auto-conversion_pytorch_single_io_model.pt', 'image_path': 'proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mlflow-deploy:v2023.3.0-main-3397', 'last_update_time': datetime.datetime(2023, 6, 23, 2, 8, 56, 669565, tzinfo=tzutc())}

## Configure model and pipeline

In [6]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [7]:
pipeline_name = f"{prefix}-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

Waiting for deployment - this will take up to 90s ................... ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.6.32',
   'name': 'engine-699fc4898f-8rv9t',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'pytorch-single-input-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'pt-single-io-model',
      'version': '8f91dee1-79e0-449b-9a59-0e93ba4a1ba9',
      'sha': '23bdbafc51c3df7ac84e5f8b2833c592d7da2b27715a7da3e45bf732ea85b8bb',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.6.31',
   'name': 'engine-lb-584f54c899-jrkg9',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.244.4.202',
   'name': 'engine-sidekick-pt-single-io-model-150-66f8c754bf-x69tx',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

## Inference

In [9]:
pipeline.infer_from_file('./data/pytorch-multi-io-data.json')

,time,in.input,out.output,check_failures
0,2023-06-23 02:09:44.784,"[0.1810392682, 0.257671871, 0.9975622994, 0.13...",[0.150135800242424],0
1,2023-06-23 02:09:44.784,"[0.6790846298, 0.0180313244, 0.5641078074, 0.3...",[-0.08427916467189789],0
2,2023-06-23 02:09:44.784,"[0.5731166617, 0.5533275412, 0.4172515078, 0.3...",[-0.01238897442817688],0
3,2023-06-23 02:09:44.784,"[0.7686341548, 0.9895859489, 0.1525565068, 0.0...",[-0.0985543504357338],0
4,2023-06-23 02:09:44.784,"[0.916929814, 0.0148056624, 0.8978319544, 0.64...",[-0.09987331926822662],0
5,2023-06-23 02:09:44.784,"[0.2164508004, 0.3161185598, 0.4462939944, 0.2...",[-0.07775364816188812],0
6,2023-06-23 02:09:44.784,"[0.3901195238, 0.4068669654, 0.370721485, 0.99...",[-0.15208657085895538],0
7,2023-06-23 02:09:44.784,"[0.4070531149, 0.1575033787, 0.9212983823, 0.7...",[-0.026245146989822388],0
8,2023-06-23 02:09:44.784,"[0.775045768, 0.4525766175, 0.7998572455, 0.85...",[-0.12369337677955627],0
9,2023-06-23 02:09:44.784,"[0.7134285141, 0.4573823559, 0.3010524343, 0.4...",[-0.03293849527835846],0


In [10]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ...................................... ok


name,pytorch-single-input-pipeline
created,2023-06-23 02:09:00.149322+00:00
last_updated,2023-06-23 02:09:00.181270+00:00
deployed,False
tags,
versions,"474221ef-b47b-41ae-9191-d156e0094ed5, 68164858-7608-4a79-a16b-d5ca3b7bfdfc"
steps,pt-single-io-model
